In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv')
test = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
train.head()

<hr>
<h2> First Task : Does Healing improve the chance of Winning</h2>


In [ ]:
task_1 = train[['heals','killPlace']]
task_1.head()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
plt.figure(figsize=(3,3))
sns.heatmap(task_1.corr() , annot=True)


Above HeatMAp clearly specify that heals are negatively corelated with probability 0.39
<hr>

<h2>Second Task : effect of Knocking (DBNO), Assisting or Reviving on Winning Percentage</h2>

1. Knocking impact on KillPlace

In [ ]:
sns.jointplot(x='DBNOs' , y='killPlace' , data=train )

2. Assisting impact on Kill Place

In [ ]:
sns.jointplot(x='assists' , y='killPlace' , data=train )

3 . Reviving impact on Kill Place

In [ ]:
sns.jointplot(x='revives' , y='killPlace' , data=train )

<hr>

<h2> Building Up the Model By applying Feature Engineering</h2>

In [ ]:
feature = train.corr()
feature

In [ ]:
feature['killPlace'].abs().sort_values(ascending=False).head(6)

<h3>Top 5 Features that is mostly co-related with killPlace:</h3>
 <ol>
    <li>killStreaks     0.810516</li>
    <li>kills           0.730498</li>
    <li>winPlacePerc    0.719069</li>
    <li>damageDealt     0.677398</li>
    <li>walkDistance    0.590236</li>
  </ol>  

In [ ]:
train.isnull().sum()

Only one Feature that have one Null Value i.e.,winPlacePerc
<h4>Fix it</h4>

In [ ]:
train['winPlacePerc'] = train['winPlacePerc'].fillna(train['winPlacePerc'].mean())
train.isnull().sum()

**Problem Got Fixed**

In [ ]:
test.isnull().sum()

In [ ]:
 x_train = train[['killStreaks','kills','damageDealt','walkDistance']].values
 x_test = test[['killStreaks','kills','damageDealt','walkDistance']].values
 y_train = train['killPlace'].values
    

In [ ]:
x_train

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.fit_transform(x_test)

In [ ]:
from sklearn import tree 

Classifier = tree.DecisionTreeClassifier(criterion = 'entropy',
                                        random_state = 0)
Classifier.fit(x_train,y_train)

In [ ]:
y_train_pred = Classifier.predict(x_train)

In [ ]:
from sklearn import metrics

metrics.accuracy_score(y_train,y_train_pred)

In [ ]:
metrics.confusion_matrix(y_train,y_train_pred)

In [ ]:
prediction = Classifier.predict(x_test)

In [ ]:
submission = pd.DataFrame({'killPlace':prediction})

#Visualize the first 5 rows
submission.head()

In [ ]:
filename = 'PUBG_Rank_Predictions_1.csv'

submission.to_csv(filename,index=False)

print('Saved file: ' + filename)